In [30]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [31]:
db = sqlite3.connect(r"DB_PATH")

In [32]:
player_data = pd.read_sql("SELECT * FROM Player;", db)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)

In [33]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [34]:
players_180_190 = len(player_data['player_api_id'].unique()[player_data['height'].between(180, 190)])
players_180_190


5925

In [35]:
players_1980 = len(player_data.query("'1980-01-01' <= birthday < '1981-01-01'"))

players_1980

434

In [36]:
highest_players = player_data.sort_values(['weight','player_name'], ascending=[False,True]).head(10)
highest_players

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
5901,5908,148325,Kristof van Hout,185306,1987-02-09 00:00:00,208.28,243
10297,10313,27313,Tim Wiese,53012,1981-12-17 00:00:00,193.04,243
4879,4884,5044,Jeroen Verhoeven,137692,1980-04-30 00:00:00,198.12,227
4408,4412,27267,Ishmael Miller,169725,1987-03-05 00:00:00,193.04,225
1595,1598,101584,Cassio,167958,1987-06-06 00:00:00,195.58,220
1819,1822,19020,Christopher Samba,46590,1984-03-28 00:00:00,193.04,220
5969,5976,210822,Lars Unnerstall,199833,1990-07-20 00:00:00,198.12,220
6762,6772,30669,Marcus Hahnemann,53785,1972-06-15 00:00:00,190.50,220
44,45,40005,Abdoulaye Faye,100329,1978-02-26 00:00:00,187.96,218
3026,3030,33060,Enoch Showunmi,157589,1982-04-21 00:00:00,195.58,218


In [37]:
s = []
for i in range(1980,1991):
    b = (i,len(player_data[pd.to_datetime(player_data['birthday']).dt.year == i]))
    s.append(b)
    
years_born_players = s 
years_born_players

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

In [38]:
a = player_data['height'][player_data.player_name.str.split(" ").str[0] == "Adriano"].mean()
b = player_data['height'][player_data.player_name.str.split(" ").str[0] == "Adriano"].std()

adriano_mean, adriano_std = a,b
adriano_mean, adriano_std



(182.11800000000002, 5.361479480723788)

In [39]:
dow_with_min_players_born = pd.to_datetime(player_data['birthday']).dt.day_name().value_counts(ascending=True).head(1).idxmin() 
dow_with_min_players_born

'Sunday'

In [40]:
league = pd.read_sql("SELECT * FROM League;",db)
matches = pd.read_sql("SELECT * FROM Match;",db)
#league['name']
a = pd.merge(league,matches,
            how='left', left_on='id', right_on='league_id')
a['name'].value_counts(ascending=False).head(1).idxmax()
league_most_matches = a['name'].value_counts(ascending=False).head(1).idxmax()
league_most_matches

              

'England Premier League'

In [41]:
a = pd.merge(player_data,matches,
            how='left', left_on='player_api_id', right_on='home_player_1')
#a['player_name'].value_counts(ascending=False).head(1).idxmax()

max_matches_player = a['player_name'].value_counts(ascending=False).head(1).idxmax()
max_matches_player

'Steve Mandanda'

In [42]:
other_cols = ["id","player_fifa_api_id","player_api_id","date",
             "preferred_food","attacking_work_rate","defensive_work_rate"]
char_cols = [c for c in player_stats_data if c not in other_cols]
corr = player_stats_data[char_cols].corr().abs().unstack().to_frame().reset_index()
corr = corr[corr['level_0'] != corr['level_1']]
corr['feature_pair'] = corr.apply(lambda x: (x['level_0'],x['level_1']), axis= 1)
corr['feature_pair'] = corr.apply(lambda x: tuple(sorted(x['feature_pair'])), axis= 1)

top_correlated_features = corr[['feature_pair',0]] \
   .drop_duplicates().sort_values(by = 0, ascending = False)[:5]['feature_pair'].values.tolist()
top_correlated_features


[('gk_positioning', 'gk_reflexes'),
 ('gk_handling', 'gk_reflexes'),
 ('gk_handling', 'gk_positioning'),
 ('sliding_tackle', 'standing_tackle'),
 ('marking', 'standing_tackle')]

In [ ]:
#!
import numpy as np

players_extended = player_stats_data \
    .merge(player_data[['player_api_id', 'player_name']], 'inner', 'player_api_id') \
    .sort_values('date').drop_duplicates(['player_api_id'], keep='last')

name = "Neymar"
neymar_vector = players_extended[players_extended['player_name'] == name][char_cols].values[0]
players_vector = players_extended[char_cols].values

players_extended['neymar_similar'] = np.sqrt(np.sum((players_vector - neymar_vector) ** 2, axis=1))
neymar_similarities = players_extended \
    .sort_values(by=['neymar_similar', 'player_name'], ascending=True)['player_name'][1:6].values.tolist()
#neymar_similarities

In [48]:
team = pd.read_sql("SELECT * FROM Team;",db)
country = pd.read_sql("SELECT * FROM Country;",db)
v = team['team_api_id'][team.team_long_name == 'Borussia Dortmund']
bor = len(matches[(matches.season =='2008/2009') & (matches.country_id == 7809) & (matches.home_team_api_id == 9789)])
borussia_bundesliga_2008_2009_matches = bor
print(borussia_bundesliga_2008_2009_matches)

17


In [51]:
match_data = pd.read_sql("SELECT * FROM Match;",db)
league_data = pd.read_sql("SELECT * FROM LEAGUE", db)
league = league_data[['id', 'name']]
league.columns = ['league_id', 'league_name']
match_league = match_data.merge(league, on=['league_id'])

home_teams = team_data[['team_api_id', 'team_long_name']] \
    .rename(columns={'team_api_id': 'home_team_api_id', 'team_long_name': 'home_team_name'})
away_teams = team_data[['team_api_id', 'team_long_name']] \
    .rename(columns={'team_api_id': 'away_team_api_id', 'team_long_name': 'away_team_name'})

match_team_league = match_league \
    .merge(home_teams, 'left', ['home_team_api_id']) \
    .merge(away_teams, 'left', ['away_team_api_id'])
match_team_target_league_season = match_team_league[(match_team_league['league_name'] == 'Germany 1. Bundesliga') &
                                    (match_team_league['season'] == '2008/2009')]

team_most_matches_bundesliga_2008_2009 = match_team_target_league_season.groupby('home_team_name').size() \
    .add(match_team_target_league_season.groupby('away_team_name').size()) \
    .sort_values(ascending=False)[0].item()
team_most_matches_bundesliga_2008_2009


34

In [52]:
def team_won(row):
    if row['home_team_goal'] > row['away_team_goal']:
        return row['home_team_name']
    elif row['home_team_goal'] < row['away_team_goal']:
        return row['away_team_name']
    else:
        return None
match_team_league['team_won'] = match_team_league.apply(lambda x: team_won(x), axis=1)

arsenal_won_matches_2015_2016 = \
    match_team_league[(match_team_league['team_won'] == 'Arsenal') & (match_team_league['season'] == '2015/2016')].shape[0]
arsenal_won_matches_2015_2016


20

In [53]:
target_matches = match_team_league[match_team_league['season'] == '2015/2016']

team_total_plays = target_matches.groupby('home_team_name').size() \
    .add(target_matches.groupby('away_team_name').size()) \
    .to_frame()

team_total_plays.columns = ['total_matches']
team_total_plays.index.names = ['team_name']

team_won_playes = target_matches.groupby('team_won').size().to_frame()
team_won_playes.columns = ['won_matches']

matches_won_total = team_total_plays.join(team_won_playes)
matches_won_total['win_rate'] = matches_won_total['won_matches'] / matches_won_total['total_matches']
team_highest_winrate_2015_2016 = matches_won_total.sort_values(['win_rate', 'team_name'], ascending=[False, True])[:1].index[0]
team_highest_winrate_2015_2016

'SL Benfica'

In [55]:
match_team_league['date'] = pd.to_datetime(match_team_league['date'])
target_league_season = match_team_league[(match_team_league['league_name'] == 'England Premier League') &
                                        (match_team_league['season'] == '2010/2011')]

team_match_date = target_league_season[['date', 'home_team_name', 'away_team_name']] \
    .melt(id_vars='date', var_name='team_type', value_name='team_name') \
    .sort_values(['team_name', 'date'])

team_match_date['prev_match_date'] = team_match_date \
    .groupby('team_name')['date'].shift(1)

team_match_date['date_diff'] = (team_match_date['date'] - team_match_date['prev_match_date']).dt.days
highest_gap_england_2010_2011 = team_match_date.sort_values('date_diff', ascending=False).iloc[0]['date_diff'].astype(int).item()
highest_gap_england_2010_2011


18

In [ ]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")